<a href="https://colab.research.google.com/github/nntadotzip/DS_coderschool_homework/blob/master/NNTA_Coderschool_DS_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [29]:
from google.colab import drive
drive.mount('/content/drive')
file = "/content/drive/My Drive/Colab Notebooks/Coderschool/labeledTrainData.tsv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [31]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df = pd.read_csv(file, sep='\t', encoding='latin-1')
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [32]:
# stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<.*?>', '', text)
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\\)|\\(|D|P|d|p)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('(?::|;|=)(?:-)?(?:\\)|\\(|D|P|d|p)', '', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ' '))
    text = re.sub('\s+', ' ', text).strip()
    return text
preprocessor('okoko <html >random :( :-D :( :P =P')

'okoko random :( : D :( :P =P'

In [0]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [0]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split
X = df.review
y = df.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [36]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words=stop, tokenizer=tokenizer_porter, preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7fc8850bd2f0>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7fc888f6a378>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

## 4. Evaluate Model

In [37]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.89      0.86      0.87      3796
           1       0.86      0.89      0.88      3704

    accuracy                           0.87      7500
   macro avg       0.88      0.88      0.87      7500
weighted avg       0.88      0.87      0.87      7500

[[3256  540]
 [ 399 3305]]


In [43]:
# playing around
test = [
    "I don't love you",
    "I hate you",
    "I :( you",
]

preds = clf.predict_proba(test)

for i in range(len(test)):
    print(f'{test[i]} --> Negative, Positive = {preds[i]}')

I don't love you --> Negative, Positive = [0.00566919 0.99433081]
I hate you --> Negative, Positive = [0.66483178 0.33516822]
I :( you --> Negative, Positive = [0.56078062 0.43921938]


## 5. Export Model 

In [0]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open(os.path.join('/content/drive/My Drive/Colab Notebooks/Coderschool/', 'hw#6.pkl'), 'wb'), protocol=4)